Code generation
===

* Symbolic function is parsed in Python
* and stored as C++ expression tree
* Can be evaluated at runtime purely within C++, and in parallel
* optionally, we can generate C++ code, compile and link in dynamically

In [ ]:
from ngsolve import *

In [ ]:
myfunc = x * sin(y)

In [ ]:
print (myfunc)

Generates a compiled instance of this function. Look for files `code*.cpp`

In [ ]:
myfunc_compiled = myfunc.Compile(realcompile=True)

In [ ]:
!ls -rtl

In [ ]:
!cat code0_0.cpp

Compiling code snippets at runtime
===

In [ ]:
def VertexPatchBlocks(fes):
    mesh = fes.mesh
    blocks = []
    freedofs = fes.FreeDofs()
    for v in mesh.vertices:
        vdofs = set()
        for el in mesh[v].elements:
            vdofs |= set(d for d in fes.GetDofNrs(el)
                         if freedofs[d])
        blocks.append(vdofs)
    return blocks

In [ ]:
from ngsolve.fem import CompilePythonModule

code = """
auto lam = [](const FESpace & fes)->shared_ptr<Table<DofId>>
{
  auto ma = fes.GetMeshAccess();
  size_t nv = ma->GetNV();
  Array<int> dofs;
  TableCreator<DofId> creator(nv+ma->GetNE());
  
  for ( ; !creator.Done(); creator++) {
    for (size_t v : Range(nv))
      creator.Add(v, v);
      
    for (size_t ed : Range(ma->GetNEdges())) {
        fes.GetDofNrs( { NT_EDGE, ed }, dofs);
        for (size_t v : ma->GetEdgePNums(ed)) 
          creator.Add(v, dofs);
    }
    
    for (size_t fa : Range(ma->GetNFaces())) {
        fes.GetDofNrs ( { NT_FACE, fa }, dofs);
        for (size_t v : ma->GetFacePNums(fa)) 
          creator.Add(v, dofs);
    }
  }
    
  return make_shared<Table<DofId>>(creator.MoveTable());
};
m.def("VertexPatchBlocks", lam);

"""

In [ ]:
try:
    m = CompilePythonModule(code)
    blockcreator = m.VertexPatchBlocks
    print ("have c++ blocks")
except:
    blockcreator = VertexPatchBlocks

In [ ]:
from ngsolve import *
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))

fes = H1(mesh, order=3)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx + u*v*dx).Assemble()


pre = Preconditioner(a, "local", blockcreator=blockcreator)
pre.Update()

In [ ]:
from ngsolve.la import EigenValues_Preconditioner
lam = EigenValues_Preconditioner(a.mat, pre.mat)
print (lam)